In [3]:
import pandas as pd
df_seoul = pd.read_csv("https://bit.ly/seoul-120-text-csv")
df_seoul[:2]

,번호,분류,제목,내용,내용번호
0,2645,복지,아빠 육아휴직 장려금,아빠 육아휴직 장려금 업무개요 남성근로자의 육아휴직을 장려하고 양육에 따른 경...,23522464
1,2644,경제,[서울산업진흥원] 서울메이드란?,서울산업진흥원 서울메이드란 서울의 감성을 담은 다양하고 새로운 경험을 제공하기 위해...,23194045


## 자연어 전처리

In [4]:
stopwords = []

In [12]:
# 토크나이징 함수 정의
from konlpy.tag import Okt
okt = Okt()

def tokenizer(raw, pos=["Noun","Alpha","Number"], stopword=stopwords):
    return [
        word for word, tag in okt.pos(
            raw, 
            stem=True    # stemming 바뀌나->바뀌다
            )
            if len(word) > 1 and tag in pos and word not in stopword

    ]

In [13]:
# tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfVectorizer = TfidfVectorizer(tokenizer=tokenizer, max_df=0.95, min_df=2)

In [14]:
features = tfidfVectorizer.fit_transform(df_seoul['내용'])
features.toarray()[:2]

/usr/local/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## 유사도확인

In [15]:
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
def similarity(x):
    result_list = cosine_similarity(features.toarray()[x:x+1], features.toarray())
    df_seoul['유사도'] = result_list[0]
    return df_seoul.sort_values(by='유사도', ascending=False)

In [17]:
similarity(1772).head(6) # 3번에 대한 유사도 검증

,번호,분류,제목,내용,내용번호,유사도
1772,873,경제,도시계획시설부지 재결신청 이후 진행단계는 어떤 과정을 거칩니까?,도시계획시설부지 재결신청 이후 진행단계는 어떤 과정을 거칩니까재결신청 이후 재결신청...,2897109,1.000000
1775,870,경제,도시계획시설 재결신청은 어떻게 합니까?,도시계획시설 재결신청은 어떻게 합니까재결신청은 소유자 및 사업시행자가 모두 할 수 ...,2897108,0.791012
1771,874,경제,도시계획시설부지의 재결과정에서는 의견을 제출할 수 없는지요?,도시계획시설부지의 재결과정에서는 의견을 제출할 수 없는지요재결신청 이후 신청서 및 ...,2897110,0.633974
1780,865,경제,도시계획시설에 편입된 보상물건의 재결신청 청구는 몇일 이내에 해야 하나요?,도시계획시설에 편입된 보상물건의 재결신청 청구는 몇일 이내에 해야 하나요사업인정 고...,2897106,0.593206
1768,877,경제,도시계획시설부지에 포함된 건물보상금액의 불만입니다.,도시계획시설부지에 포함된 건물보상금액의 불만입니다사업시행자로 하여금 재결신청을 청구...,2897114,0.567231
1767,878,경제,도시계획시설 부지의 이의재결이란 무엇입니까?,도시계획시설 부지의 이의재결이란 무엇입니까수용재결에 대한 이의가 있는 경우 중앙토지...,2897115,0.539866


## 확인

In [18]:
import pandas as pd
import pyLDAvis
import pyLDAvis.lda_model
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups

### n_components = 3 

In [55]:
components_3  = LatentDirichletAllocation(n_components=3, n_jobs=-1) # n_components => topic 가짓수 결정 / n_jobs =>성능높이기

In [56]:
components_3.fit(features)

LatentDirichletAllocation(n_components=3, n_jobs=-1)

In [59]:
vis = pyLDAvis.lda_model.prepare(components_3, features, tfidfVectorizer) # 토픽모델, 교육이 끝난 값(행렬형태), 교육모델

In [60]:
pyLDAvis.enable_notebook()
pyLDAvis.display(vis) # PCA - 차원축소

### n_components = 5

In [61]:
components_5 = LatentDirichletAllocation(n_components=5, n_jobs=-1) # n_components => topic 가짓수 결정 / n_jobs =>성능높이기

In [62]:
components_5.fit(features)

LatentDirichletAllocation(n_components=5, n_jobs=-1)

In [63]:
vis = pyLDAvis.lda_model.prepare(components_5, features, tfidfVectorizer) # 토픽모델, 교육이 끝난 값(행렬형태), 교육모델

In [64]:
pyLDAvis.enable_notebook()
pyLDAvis.display(vis) # PCA - 차원축소

### n_components = 7

In [65]:
components_7 = LatentDirichletAllocation(n_components=7, n_jobs=-1) # n_components => topic 가짓수 결정 / n_jobs =>성능높이기

In [66]:
components_7.fit(features)

LatentDirichletAllocation(n_components=7, n_jobs=-1)

In [67]:
vis = pyLDAvis.lda_model.prepare(components_7, features, tfidfVectorizer) # 토픽모델, 교육이 끝난 값(행렬형태), 교육모델

In [68]:
pyLDAvis.enable_notebook()
pyLDAvis.display(vis) # PCA - 차원축소